In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Sergipe - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Sergipe - IDH,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH Educacao,Sergipe - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Desemprego,Sergipe - IDH Renda,Sergipe - Consumo de Cimento (t)
0,2014,0.690442,2.470591e+07,1.254702e+06,9.861955,2.051042e+07,0.637414,3058.777409,5.676413e+08,2.779340e+09,...,20.010392,1014.355116,1389.188275,26.773639,19.486560,67.782988,2.799381,13.099831,0.668238,633.190143
1,2015,0.700339,2.484109e+07,1.238716e+06,9.830307,2.046045e+07,0.650007,2732.441149,5.179392e+08,2.561731e+09,...,18.966088,1014.406517,1396.519093,26.838598,19.578296,66.497486,2.216394,13.605605,0.673638,592.014514
2,2016,0.700643,2.492621e+07,1.222457e+06,9.814014,2.043197e+07,0.651553,2388.888256,4.524144e+08,2.516859e+09,...,18.086986,1014.355535,1396.331304,26.770657,19.675461,66.457655,1.943788,13.929548,0.673060,552.624343
3,2017,0.700974,2.497554e+07,1.205785e+06,9.822126,2.044513e+07,0.653967,2146.114260,4.213689e+08,2.497710e+09,...,18.116372,1014.300674,1352.042945,26.799279,19.803363,66.478178,2.248768,13.725374,0.672156,473.789000
4,2018,0.701153,2.493468e+07,1.197917e+06,9.829110,2.044167e+07,0.655327,2061.548731,4.166580e+08,2.797145e+09,...,18.626394,1014.188575,1309.565375,26.901325,19.661311,65.129851,2.409893,13.638962,0.672352,372.210000
5,2019,0.700349,2.481931e+07,1.201063e+06,9.837051,2.044502e+07,0.654902,2120.688018,4.462625e+08,3.484252e+09,...,19.239778,1014.054024,1312.424512,26.912623,18.049894,61.281496,2.441780,13.518614,0.672021,409.903000
6,2020,0.699926,2.455519e+07,1.212774e+06,9.838051,2.041845e+07,0.656348,2142.762308,5.489412e+08,4.114911e+09,...,19.018643,1014.607943,1340.672407,26.896442,16.666837,58.944410,2.433043,13.428525,0.671114,526.396000
7,2021,0.697919,2.418981e+07,1.225824e+06,9.859826,2.041650e+07,0.654016,2078.152252,6.638811e+08,4.421848e+09,...,19.325160,1015.158297,1368.237270,26.821190,16.635910,59.136794,2.424321,13.354176,0.669669,511.494000
8,2022,0.697548,2.374170e+07,1.240388e+06,9.883407,2.040661e+07,0.651290,2043.813741,7.010332e+08,4.533216e+09,...,19.585864,1015.209022,1389.692135,26.758169,16.802998,59.053802,2.427658,13.374761,0.670367,425.370000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Sergipe - IDH,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH Educacao,Sergipe - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe -IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Desemprego,Sergipe - IDH Renda
0,0.690442,2.470591e+07,1.254702e+06,9.861955,2.051042e+07,0.637414,3058.777409,5.676413e+08,2.779340e+09,1.954361e+10,...,0.772499,20.010392,1014.355116,1389.188275,26.773639,19.486560,67.782988,2.799381,13.099831,0.668238
1,0.700339,2.484109e+07,1.238716e+06,9.830307,2.046045e+07,0.650007,2732.441149,5.179392e+08,2.561731e+09,1.327596e+10,...,0.784877,18.966088,1014.406517,1396.519093,26.838598,19.578296,66.497486,2.216394,13.605605,0.673638
2,0.700643,2.492621e+07,1.222457e+06,9.814014,2.043197e+07,0.651553,2388.888256,4.524144e+08,2.516859e+09,1.011370e+10,...,0.784594,18.086986,1014.355535,1396.331304,26.770657,19.675461,66.457655,1.943788,13.929548,0.673060
3,0.700974,2.497554e+07,1.205785e+06,9.822126,2.044513e+07,0.653967,2146.114260,4.213689e+08,2.497710e+09,8.456309e+09,...,0.783829,18.116372,1014.300674,1352.042945,26.799279,19.803363,66.478178,2.248768,13.725374,0.672156
4,0.701153,2.493468e+07,1.197917e+06,9.829110,2.044167e+07,0.655327,2061.548731,4.166580e+08,2.797145e+09,7.937760e+09,...,0.782632,18.626394,1014.188575,1309.565375,26.901325,19.661311,65.129851,2.409893,13.638962,0.672352
5,0.700349,2.481931e+07,1.201063e+06,9.837051,2.044502e+07,0.654902,2120.688018,4.462625e+08,3.484252e+09,1.091282e+10,...,0.780962,19.239778,1014.054024,1312.424512,26.912623,18.049894,61.281496,2.441780,13.518614,0.672021
6,0.699926,2.455519e+07,1.212774e+06,9.838051,2.041845e+07,0.656348,2142.762308,5.489412e+08,4.114911e+09,1.625333e+10,...,0.778777,19.018643,1014.607943,1340.672407,26.896442,16.666837,58.944410,2.433043,13.428525,0.671114


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    592.014514
1    552.624343
2    473.789000
3    372.210000
4    409.903000
5    526.396000
6    511.494000
7    425.370000
8           NaN
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Sergipe - IDH,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH Educacao,Sergipe - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe -IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Desemprego,Sergipe - IDH Renda
0,-2.229244,-1.788251,1.659093,1.965670,2.115653,-2.130730,1.754972,1.779974,0.019134,2.003355,...,-2.127519,1.749121,0.650346,0.798855,-1.033487,0.184345,1.047506,1.751450,-1.929215,-2.116954
1,0.353859,-0.288828,0.892445,-0.141125,0.180888,-0.084754,0.861090,0.870373,-0.621387,0.401153,...,0.777147,0.187098,1.076862,0.995090,0.103458,0.337047,0.429353,-0.487514,0.076468,0.995590
2,0.433086,0.655446,0.112729,-1.225740,-0.921810,0.166444,-0.079950,-0.328802,-0.753466,-0.407217,...,0.710534,-1.127822,0.653821,0.990064,-1.085675,0.498785,0.410200,-1.534458,1.361086,0.662253
3,0.519515,1.202629,-0.686826,-0.685772,-0.412176,0.558644,-0.744943,-0.896969,-0.809829,-0.830898,...,0.531202,-1.083868,0.198598,-0.195472,-0.584710,0.711689,0.420069,-0.363181,0.551418,0.141418
4,0.566346,0.749357,-1.064143,-0.220827,-0.546133,0.779708,-0.976580,-0.983185,0.071542,-0.963455,...,0.250237,-0.321000,-0.731577,-1.332536,1.201336,0.475232,-0.228295,0.255620,0.208744,0.254409
5,0.356437,-0.530354,-0.913298,0.307794,-0.416421,0.710687,-0.814589,-0.441390,2.094006,-0.202939,...,-0.141601,0.596472,-1.848050,-1.256001,1.399077,-2.207098,-2.078833,0.378083,-0.268501,0.063283


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    592.014514
1    552.624343
2    473.789000
3    372.210000
4    409.903000
5    526.396000
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Sergipe - IDH,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH Educacao,Sergipe - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe -IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Desemprego,Sergipe - IDH Renda
6,0.226848,-1.945833,-0.322727,0.343224,-1.174161,0.824365,-0.671441,1.16956,2.035192,0.985175,...,-0.588126,0.244766,1.764644,-0.454723,0.9519,-2.112686,-1.88844,0.3163,-0.563785,-0.419233


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    511.494
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [14]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[323760497, 2512765463, 3780143421, 177257787, 3940965160, 3565817477, 2435575019, 931518497, 579022924, 3060916225]


Step: 0 ___________________________________________


2023-08-20 12:32:44.711887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-20 12:32:44.712872: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1344: early stopping
val_loss: 1276.80810546875
winner_seed: 323760497


Step: 1 ___________________________________________
Epoch 1185: early stopping
val_loss: 5643.75634765625


Step: 2 ___________________________________________
Epoch 2849: early stopping
val_loss: 539.4278564453125
winner_seed: 3780143421


Step: 3 ___________________________________________
Epoch 1380: early stopping
val_loss: 7673.32275390625


Step: 4 ___________________________________________
Epoch 4471: early stopping
val_loss: 687.0084228515625


Step: 5 ___________________________________________
Epoch 1704: early stopping
val_loss: 452.19305419921875
winner_seed: 3565817477


Step: 6 ___________________________________________
Epoch 1598: early stopping
val_loss: 43.42443084716797
winner_seed: 2435575019


Step: 7 ___________________________________________
Epoch 1348: early stopping
val_loss: 764.3483276367188


Step: 8 ___________________________________________
Epoch 1503: early stopping
val_loss

In [15]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 1s 527ms/step - loss: 244221.9219 - val_loss: 258570.5781
Epoch 2/5000
1/1 [==============================] - 0s 38ms/step - loss: 232315.6250 - val_loss: 293849.6250
Epoch 3/5000
1/1 [==============================] - 0s 39ms/step - loss: 207148.0625 - val_loss: 220964.1250
Epoch 4/5000
1/1 [==============================] - 0s 43ms/step - loss: 189397.2969 - val_loss: 294593.4062
Epoch 5/5000
1/1 [==============================] - 0s 41ms/step - loss: 173771.2031 - val_loss: 187599.7188
Epoch 6/5000
1/1 [==============================] - 0s 33ms/step - loss: 179742.2500 - val_loss: 513516.9375
Epoch 7/5000
1/1 [==============================] - 0s 38ms/step - loss: 208603.6406 - val_loss: 213103.3906
Epoch 8/5000
1/1 [==============================] - 0s 34ms/step - loss: 179079.3281 - val_loss: 214426.0938
Epoch 9/5000
1/1 [==============================] - 0s 33ms/step - loss: 181447.7344 - val_loss: 264509.3125
Epoch 10/5000
1/1 

1/1 [==============================] - 0s 38ms/step - loss: 20822.3555 - val_loss: 33410.5820
Epoch 77/5000
1/1 [==============================] - 0s 37ms/step - loss: 18907.8730 - val_loss: 37903.3398
Epoch 78/5000
1/1 [==============================] - 0s 47ms/step - loss: 18451.3770 - val_loss: 35011.5742
Epoch 79/5000
1/1 [==============================] - 0s 34ms/step - loss: 17821.7188 - val_loss: 35150.9961
Epoch 80/5000
1/1 [==============================] - 0s 34ms/step - loss: 16970.3770 - val_loss: 36212.6680
Epoch 81/5000
1/1 [==============================] - 0s 35ms/step - loss: 17532.7598 - val_loss: 31063.5898
Epoch 82/5000
1/1 [==============================] - 0s 36ms/step - loss: 16229.0576 - val_loss: 34202.3867
Epoch 83/5000
1/1 [==============================] - 0s 45ms/step - loss: 15349.0234 - val_loss: 35585.0312
Epoch 84/5000
1/1 [==============================] - 0s 36ms/step - loss: 16676.0957 - val_loss: 37864.2578
Epoch 85/5000
1/1 [=======================

Epoch 152/5000
1/1 [==============================] - 0s 34ms/step - loss: 3777.4543 - val_loss: 15902.0508
Epoch 153/5000
1/1 [==============================] - 0s 35ms/step - loss: 3690.3857 - val_loss: 14263.0205
Epoch 154/5000
1/1 [==============================] - 0s 36ms/step - loss: 3730.1018 - val_loss: 15317.5674
Epoch 155/5000
1/1 [==============================] - 0s 35ms/step - loss: 3803.2461 - val_loss: 7619.2207
Epoch 156/5000
1/1 [==============================] - 0s 36ms/step - loss: 4498.4766 - val_loss: 15333.8125
Epoch 157/5000
1/1 [==============================] - 0s 35ms/step - loss: 4969.6616 - val_loss: 10767.7803
Epoch 158/5000
1/1 [==============================] - 0s 34ms/step - loss: 5119.1050 - val_loss: 11344.5967
Epoch 159/5000
1/1 [==============================] - 0s 34ms/step - loss: 3522.9021 - val_loss: 14555.7070
Epoch 160/5000
1/1 [==============================] - 0s 37ms/step - loss: 3350.9714 - val_loss: 13857.5605
Epoch 161/5000
1/1 [=========

1/1 [==============================] - 0s 35ms/step - loss: 2079.1467 - val_loss: 8430.1572
Epoch 229/5000
1/1 [==============================] - 0s 37ms/step - loss: 2048.1206 - val_loss: 8441.2344
Epoch 230/5000
1/1 [==============================] - 0s 41ms/step - loss: 2022.7218 - val_loss: 8340.2129
Epoch 231/5000
1/1 [==============================] - 0s 37ms/step - loss: 2150.5388 - val_loss: 8849.8555
Epoch 232/5000
1/1 [==============================] - 0s 35ms/step - loss: 2131.3054 - val_loss: 8708.2021
Epoch 233/5000
1/1 [==============================] - 0s 35ms/step - loss: 1972.4653 - val_loss: 8077.3511
Epoch 234/5000
1/1 [==============================] - 0s 35ms/step - loss: 1944.3844 - val_loss: 8099.3794
Epoch 235/5000
1/1 [==============================] - 0s 36ms/step - loss: 1922.6056 - val_loss: 8002.5713
Epoch 236/5000
1/1 [==============================] - 0s 36ms/step - loss: 2160.6980 - val_loss: 5933.8887
Epoch 237/5000
1/1 [==============================] 

1/1 [==============================] - 0s 36ms/step - loss: 1405.1738 - val_loss: 5177.1689
Epoch 305/5000
1/1 [==============================] - 0s 36ms/step - loss: 1324.0636 - val_loss: 5423.7329
Epoch 306/5000
1/1 [==============================] - 0s 36ms/step - loss: 1314.5234 - val_loss: 5339.2886
Epoch 307/5000
1/1 [==============================] - 0s 35ms/step - loss: 1307.4202 - val_loss: 5314.5161
Epoch 308/5000
1/1 [==============================] - 0s 39ms/step - loss: 1300.7831 - val_loss: 5279.8442
Epoch 309/5000
1/1 [==============================] - 0s 36ms/step - loss: 1536.2389 - val_loss: 6033.6880
Epoch 310/5000
1/1 [==============================] - 0s 34ms/step - loss: 1314.0916 - val_loss: 5108.6328
Epoch 311/5000
1/1 [==============================] - 0s 33ms/step - loss: 1284.9949 - val_loss: 5230.4570
Epoch 312/5000
1/1 [==============================] - 0s 38ms/step - loss: 1278.5792 - val_loss: 5168.8809
Epoch 313/5000
1/1 [==============================] 

...

1/1 [==============================] - 0s 36ms/step - loss: 1164.7100 - val_loss: 3391.1074
Epoch 1220/5000
1/1 [==============================] - 0s 38ms/step - loss: 777.4807 - val_loss: 2831.1311
Epoch 1221/5000
1/1 [==============================] - 0s 38ms/step - loss: 1174.6288 - val_loss: 3533.6907
Epoch 1222/5000
1/1 [==============================] - 0s 35ms/step - loss: 1157.5649 - val_loss: 3205.7236
Epoch 1223/5000
1/1 [==============================] - 0s 36ms/step - loss: 1150.9279 - val_loss: 3560.3137
Epoch 1224/5000
1/1 [==============================] - 0s 34ms/step - loss: 1150.1516 - val_loss: 3449.4114
Epoch 1225/5000
1/1 [==============================] - 0s 35ms/step - loss: 1146.2950 - val_loss: 3471.6506
Epoch 1226/5000
1/1 [==============================] - 0s 33ms/step - loss: 1183.7513 - val_loss: 3483.4060
Epoch 1227/5000
1/1 [==============================] - 0s 34ms/step - loss: 1246.8651 - val_loss: 3976.6287
Epoch 1228/5000
1/1 [========================

1/1 [==============================] - 0s 34ms/step - loss: 544.1472 - val_loss: 3641.6555
Epoch 1296/5000
1/1 [==============================] - 0s 34ms/step - loss: 789.9566 - val_loss: 2724.1851
Epoch 1297/5000
1/1 [==============================] - 0s 33ms/step - loss: 726.5559 - val_loss: 2636.9893
Epoch 1298/5000
1/1 [==============================] - 0s 35ms/step - loss: 1198.0870 - val_loss: 3284.5513
Epoch 1299/5000
1/1 [==============================] - 0s 33ms/step - loss: 1190.2252 - val_loss: 3343.1709
Epoch 1300/5000
1/1 [==============================] - 0s 35ms/step - loss: 1175.6136 - val_loss: 3346.7151
Epoch 1301/5000
1/1 [==============================] - 0s 38ms/step - loss: 1150.3385 - val_loss: 3343.9651
Epoch 1302/5000
1/1 [==============================] - 0s 47ms/step - loss: 1163.3296 - val_loss: 3371.1406
Epoch 1303/5000
1/1 [==============================] - 0s 39ms/step - loss: 1146.3766 - val_loss: 3374.6001
Epoch 1304/5000
1/1 [==========================

1/1 [==============================] - 0s 35ms/step - loss: 1156.2301 - val_loss: 3292.1919
Epoch 1372/5000
1/1 [==============================] - 0s 44ms/step - loss: 933.5582 - val_loss: 3026.8567
Epoch 1373/5000
1/1 [==============================] - 0s 45ms/step - loss: 919.8584 - val_loss: 2977.8591
Epoch 1374/5000
1/1 [==============================] - 0s 38ms/step - loss: 913.3243 - val_loss: 2923.0740
Epoch 1375/5000
1/1 [==============================] - 0s 36ms/step - loss: 1431.0773 - val_loss: 4144.5220
Epoch 1376/5000
1/1 [==============================] - 0s 34ms/step - loss: 1333.0193 - val_loss: 4154.1816
Epoch 1377/5000
1/1 [==============================] - 0s 34ms/step - loss: 1190.0736 - val_loss: 3660.0432
Epoch 1378/5000
1/1 [==============================] - 0s 34ms/step - loss: 1169.4507 - val_loss: 3270.1204
Epoch 1379/5000
1/1 [==============================] - 0s 35ms/step - loss: 909.4602 - val_loss: 2910.6138
Epoch 1380/5000
1/1 [===========================

1/1 [==============================] - 0s 38ms/step - loss: 1145.5492 - val_loss: 3512.5083
Epoch 1448/5000
1/1 [==============================] - 0s 40ms/step - loss: 1177.9271 - val_loss: 3483.6113
Epoch 1449/5000
1/1 [==============================] - 0s 41ms/step - loss: 1353.6764 - val_loss: 4323.7407
Epoch 1450/5000
1/1 [==============================] - 0s 35ms/step - loss: 1180.3239 - val_loss: 3502.2751
Epoch 1451/5000
1/1 [==============================] - 0s 38ms/step - loss: 1138.8157 - val_loss: 3488.3250
Epoch 1452/5000
1/1 [==============================] - 0s 34ms/step - loss: 1137.4169 - val_loss: 3502.2390
Epoch 1453/5000
1/1 [==============================] - 0s 36ms/step - loss: 1136.5851 - val_loss: 3514.4836
Epoch 1454/5000
1/1 [==============================] - 0s 40ms/step - loss: 1140.5422 - val_loss: 3526.9817
Epoch 1455/5000
1/1 [==============================] - 0s 34ms/step - loss: 1182.5193 - val_loss: 3492.3923
Epoch 1456/5000
1/1 [=======================

1/1 [==============================] - 0s 46ms/step - loss: 1135.5975 - val_loss: 3586.2876
Epoch 1524/5000
1/1 [==============================] - 0s 46ms/step - loss: 1134.8213 - val_loss: 3583.2178
Epoch 1525/5000
1/1 [==============================] - 0s 46ms/step - loss: 1165.7325 - val_loss: 3539.6255
Epoch 1526/5000
1/1 [==============================] - 0s 36ms/step - loss: 1151.5127 - val_loss: 3567.5320
Epoch 1527/5000
1/1 [==============================] - 0s 48ms/step - loss: 1153.6631 - val_loss: 3629.9375
Epoch 1528/5000
1/1 [==============================] - 0s 45ms/step - loss: 1145.5072 - val_loss: 3606.4446
Epoch 1529/5000
1/1 [==============================] - 0s 44ms/step - loss: 1134.6703 - val_loss: 3574.5276
Epoch 1530/5000
1/1 [==============================] - 0s 37ms/step - loss: 1260.4408 - val_loss: 3828.5911
Epoch 1531/5000
1/1 [==============================] - 0s 48ms/step - loss: 1138.2313 - val_loss: 3610.1768
Epoch 1532/5000
1/1 [=======================

In [16]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 88ms/step
[511.494](test_target) - [[489.293]](prediction) = 22.201000732421903


In [17]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [18]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Sergipe - IDH,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH Educacao,Sergipe - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe -IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Desemprego,Sergipe - IDH Renda
0,-1.413730,-1.306301,1.22128,1.332905,1.320169,-1.407170,1.214119,1.165029,1.396038,1.335266,...,-1.413928,1.258264,-0.715764,-1.413857,-0.659016,-1.212846,1.413720,1.343689,-1.303817,-1.407472
1,0.674836,0.183913,0.00690,-0.257157,-0.220921,0.581504,0.020982,0.111769,-0.502296,-0.264151,...,0.731566,-0.070044,1.414178,0.734444,1.413147,-0.023460,-0.674509,-0.289908,0.177519,0.823182
2,0.738894,1.122388,-1.22818,-1.075748,-1.099248,0.825666,-1.235101,-1.276798,-0.893743,-1.071116,...,0.682363,-1.188220,-0.698414,0.679412,-0.754131,1.236306,-0.739212,-1.053781,1.126298,0.584290


train_target:


,Sergipe - Consumo de Cimento (t)
0,592.014514
1,552.624343
2,473.789000


test_input:


...

,Sergipe - IDH,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH Educacao,Sergipe - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe -IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Desemprego,Sergipe - IDH Renda
6,0.226848,-1.945833,-0.322727,0.343224,-1.174161,0.824365,-0.671441,1.16956,2.035192,0.985175,...,-0.588126,0.244766,1.764644,-0.454723,0.9519,-2.112686,-1.88844,0.3163,-0.563785,-0.419233


test_target:


,Sergipe - Consumo de Cimento (t)
6,511.494


Epoch 1598: early stopping
1/1 [==============================] - 0s 81ms/step
Error: 22.201000732421903




[513.207763671875, 422.9987487792969, 391.08746337890625, 489.2929992675781]

In [19]:
display(targets)
display(predictions)

[372.21, 409.903, 526.396, 511.494]

[513.207763671875, 422.9987487792969, 391.08746337890625, 489.2929992675781]

In [20]:
mae = mean_absolute_error(predictions, targets)
mae

77.90076245117187

In [21]:
porcentage = mae/np.mean(targets)
porcentage

0.17121018471106228